In [ ]:
import pandas as pd
import numpy as np
import ortega
from math import radians,cos,sin,asin,sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.stats as stats
from sklearn.mixture import GaussianMixture
from scipy.optimize import minimize

from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkt
from scipy.spatial import ConvexHull

import geopandas as gpd
import rasterio
from rasterio.mask import mask

from scipy.stats import mannwhitneyu
from pathlib import Path

# !pip install --upgrade ortega
# !pip install pyarrow

# Read in GPS data

In [ ]:
df = pd.read_csv('../../data/tiger_leopard_env.csv')
df['Time_LMT'] = pd.to_datetime(df['Time_LMT'])
df.groupby(['idcollar'])[['slope', 'sb', 'bt']].agg(['min','max','count', 'mean'])

In [3]:
df.groupby(['idcollar'])[['Time_LMT']].agg(['min','max','count'])

Time_LMT                           
                         min                 max  count
idcollar                                               
31898    2018-09-27 22:15:00 2020-02-18 06:02:00  10717
31899    2019-03-10 21:30:00 2020-02-22 12:01:00   8034
37821    2019-10-09 10:00:00 2020-03-28 03:00:00   3470
37822    2019-10-16 07:00:00 2020-03-07 09:01:00   2257
37823    2019-10-19 23:00:00 2021-01-06 15:42:00   9507
131343   2018-08-30 09:00:00 2020-11-16 07:00:00  18584
229011   2017-12-20 07:00:00 2019-01-30 19:00:00   9504
229012   2019-12-21 08:01:00 2020-12-28 06:00:00   7164
229022   2019-11-04 10:00:00 2020-12-09 15:00:00   7786
229032   2019-09-12 22:00:00 2020-10-02 02:00:00   8625
229041   2018-08-20 20:00:00 2019-09-22 08:02:00   6151

In [4]:
df.columns

Index(['ID', 'event_id', 'idcollar', 'latitude', 'longitude', 'DSM', 'slope',
       'temperature', 'Time_UTC', 'Time_LMT', 'stepLength', 'duration',
       'speed', 'year', 'month', 'day', 'hour', 'daynight', 'drywet',
       'year/month', 'proj_lon', 'proj_lat', 'stream', 'road_trail', 'trail',
       'road', 'village', 'NDVI', 'altitude', 'tg', 'sb', 'gr', 'bt', 'prey',
       'Behavior_label'],
      dtype='object')

# Compute distribution

In [5]:
def compute_turning_angles(df):
    """
    Computes turning angles for each individual and adds them to the dataframe.
    """
    df = df.copy()  # Avoid modifying the original dataframe

    # Compute displacement vectors
    df['dx'] = df.groupby('idcollar')['proj_lon'].diff()
    df['dy'] = df.groupby('idcollar')['proj_lat'].diff()

    # Compute movement directions (angles in radians)
    df['direction'] = np.arctan2(df['dy'], df['dx'])

    # Compute turning angles (change in direction between consecutive steps)
    df['turning_angle'] = df.groupby('idcollar')['direction'].diff()

    # Normalize turning angles to range [-π, π]
    df['turning_angle'] = df['turning_angle'].apply(lambda x: (x + np.pi) % (2 * np.pi) - np.pi)

    return df

In [6]:
df = compute_turning_angles(df)  # Add turning angle column
df.to_csv('../../data/tiger_leopard_env_tg.csv')
print("Turning angles computed and saved to updated CSV file.")

Turning angles computed and saved to updated CSV file.


In [7]:
df.groupby(['idcollar'])[['stepLength', 'turning_angle']].agg(['min','max','count', 'mean', 'median'])

stepLength                                       turning_angle  \
                min        max  count      mean    median           min   
idcollar                                                                  
31898           0.0   7.225146  10717  0.143853  0.022400     -3.141593   
31899           0.0   3.060886   8034  0.149462  0.025398     -3.141593   
37821           0.0   8.330141   3470  0.114664  0.018964     -3.141593   
37822           0.0   8.296052   2257  0.238676  0.041692     -3.141593   
37823           0.0   9.557510   9507  0.308503  0.044157     -3.141593   
131343          0.0   8.586030  18584  0.281316  0.038141     -3.141593   
229011          0.0  16.692342   9504  0.142698  0.018955     -3.141593   
229012          0.0  15.289849   7164  0.160584  0.021323     -3.141583   
229022          0.0  11.916479   7786  0.281002  0.026128     -3.141593   
229032          0.0   4.176556   8625  0.213572  0.029240     -3.141593   
229041          0.0  19.419672   6151  0.391094  0.029653     -3.141593   

                                               
               max  count      mean    median  
idcollar                                       
31898     3.141593  10715 -0.020977 -0.033245  
31899     3.141178   8032  0.037870  0.047588  
37821     3.139853   3468 -0.002083 -0.038864  
37822     3.140194   2255  0.000239  0.047619  
37823     3.140023   9505 -0.020415 -0.025846  
131343    3.141294  18582  0.035077  0.040854  
229011    3.141526   9502 -0.015796 -0.021074  
229012    3.141530   7162  0.001342 -0.007033  
229022    3.141533   7784  0.015167 -0.001848  
229032    3.141446   8623 -0.021255 -0.036740  
229041    3.141226   6149  0.032170  0.040875

In [8]:
def extract_adjusted_tracking_data(file_path, id_pair, output_path):
    """
    Extracts the tracking data within the adjusted shared timeline.
    - Keeps three weeks before the shared start time for early starters (if possible).
    - Keeps three weeks after the shared end time for late finishers (if possible).
    - If the time difference is less than three weeks, keeps the original data.
    """
    # Load the dataset
    df = pd.read_csv(file_path)

    # Convert time column to datetime format
    df['Time_LMT'] = pd.to_datetime(df['Time_LMT'])

    # Filter for the selected individuals
    df_pair = df[df['idcollar'].isin(id_pair)].copy()

    # Find the earliest and latest tracking times for each individual
    earliest_times = df_pair.groupby('idcollar')['Time_LMT'].min()
    latest_times = df_pair.groupby('idcollar')['Time_LMT'].max()

    # Determine the shared start and end times
    shared_earliest_time = earliest_times.max()
    shared_latest_time = latest_times.min()

    # Apply a three-week buffer for early starters
    adjusted_start_times = {}
    for id in id_pair:
        individual_start = earliest_times[id]
        if (shared_earliest_time - individual_start) > pd.Timedelta(weeks=0):
            adjusted_start_times[id] = shared_earliest_time - pd.Timedelta(weeks=0)
        else:
            adjusted_start_times[id] = individual_start  # Keep original if within three weeks

    # Apply a three-week buffer for late finishers
    adjusted_end_times = {}
    for id in id_pair:
        individual_end = latest_times[id]
        if (individual_end - shared_latest_time) > pd.Timedelta(weeks=0):
            adjusted_end_times[id] = shared_latest_time + pd.Timedelta(weeks=0)
        else:
            adjusted_end_times[id] = individual_end  # Keep original if within three weeks

    # Extract rows within the adjusted timeline for each individual
    df_filtered = pd.concat([
        df_pair[(df_pair['idcollar'] == id) &
                (df_pair['Time_LMT'] >= adjusted_start_times[id]) &
                (df_pair['Time_LMT'] <= adjusted_end_times[id])]
        for id in id_pair
    ])

    # Sort by time for proper sequence
    df_filtered = df_filtered.sort_values(by=['Time_LMT'])

    # Save the extracted dataset
    df_filtered[['idcollar', 'Time_LMT', 'proj_lon', 'proj_lat', 'slope', 'sb', 'bt', 'gr']].to_csv(
        output_path, index=False
    )

    print(f"Extracted tracking data saved to {output_path}")

    return df_filtered  # Return the filtered DataFrame

In [9]:
def analyze_step_length_distribution(df, id_pair):
    """
    Analyzes the step length distribution for a given pair of animals.
    Fits multiple probability distributions and selects the best one.

    """
    # Filter dataset for the selected individuals
    df_pair = df[df['idcollar'].isin(id_pair)]

    # Define distributions to test
    distributions = [stats.gamma, stats.lognorm, stats.expon, stats.weibull_min]

    # Store results
    best_fits = {}

    # Plot step length distributions and fit different distributions
    plt.figure(figsize=(12, 6))

    for i, id in enumerate(id_pair):
        step_lengths = df_pair[df_pair['idcollar'] == id]['stepLength'].dropna()

        # Histogram of step lengths
        plt.subplot(1, 2, i + 1)
        plt.hist(step_lengths, bins=30, density=True, alpha=0.6, color='gray', label='Observed')

        # Fit each distribution and compute log-likelihood
        best_fit = None
        best_ll = -np.inf
        best_params = None

        for dist in distributions:
            params = dist.fit(step_lengths)  # Fit the distribution
            ll = np.sum(dist.logpdf(step_lengths, *params))  # Compute log-likelihood

            # Check if this is the best fit
            if ll > best_ll:
                best_ll = ll
                best_fit = dist
                best_params = params

        # Store best fit
        best_fits[id] = {"distribution": best_fit.name, "parameters": best_params}

        # Plot best-fitting distribution
        x = np.linspace(min(step_lengths), max(step_lengths), 100)
        y = best_fit.pdf(x, *best_params)
        plt.plot(x, y, label=f'Best fit: {best_fit.name}', linewidth=2)

        plt.title(f'Step Length Distribution - {id}')
        plt.xlabel('Step Length (km)')
        plt.ylabel('Density')
        plt.legend()

        # Print best-fitting distribution parameters
        print(f"Best fit for {id}: {best_fit.name}")
        print(f"Parameters: {best_params}")

    plt.tight_layout()
    plt.show()

    return best_fits

In [10]:
def fit_mixture_wrapped_normal(angles):
    """
    Fits a mixture of two wrapped normal distributions to capture bimodality.

    Parameters:
    - angles: np.array, observed turning angles

    Returns:
    - best_params: list of fitted wrapped normal parameters
    """
    def wrapped_normal_mixture_neg_log_likelihood(params):
        """Negative log-likelihood function for a mixture of two wrapped normal distributions."""
        w, mu1, sigma1, mu2, sigma2 = params
        w = np.clip(w, 0.01, 0.99)  # Ensure valid mixture weights
        likelihood = (
            w * stats.vonmises.pdf(angles, sigma1, loc=mu1) +
            (1 - w) * stats.vonmises.pdf(angles, sigma2, loc=mu2)
        )
        return -np.sum(np.log(likelihood + 1e-9))  # Avoid log(0)

    # Initial guesses: one component near -π, one near +π
    initial_params = [0.5, -np.pi, 2, np.pi, 2]  # w, mu1, sigma1, mu2, sigma2
    bounds = [(0.01, 0.99), (-np.pi, np.pi), (0.1, 10), (-np.pi, np.pi), (0.1, 10)]

    # Optimize parameters using MLE
    result = minimize(wrapped_normal_mixture_neg_log_likelihood, initial_params, bounds=bounds)
    best_params = result.x

    return best_params

In [11]:
def calculate_turning_angles(df, id_pair):
    """
    Computes turning angles for a given pair of individuals.
    """
    df_pair = df[df['idcollar'].isin(id_pair)].copy()

    # Compute displacement vectors
    df_pair['dx'] = df_pair.groupby('idcollar')['proj_lon'].diff()
    df_pair['dy'] = df_pair.groupby('idcollar')['proj_lat'].diff()

    # Compute movement directions (angles in radians)
    df_pair['direction'] = np.arctan2(df_pair['dy'], df_pair['dx'])

    # Compute turning angles (change in direction between consecutive steps)
    df_pair['turning_angle'] = df_pair.groupby('idcollar')['direction'].diff()

    # Normalize turning angles to range [-π, π]
    df_pair['turning_angle'] = df_pair['turning_angle'].apply(lambda x: (x + np.pi) % (2 * np.pi) - np.pi)

    return df_pair.dropna(subset=['turning_angle'])

In [12]:
def analyze_turning_angle_distribution(df, id_pair):
    """
    Analyzes the turning angle distribution for a given pair of animals.
    Fits a Mixture of Two Wrapped Normal Distributions.

    Parameters:
    - df: pandas DataFrame containing tracking data with 'turning_angle' column
    - id_pair: list of two idcollar values (e.g., [37821, 229032])

    Returns:
    - Dictionary with the best-fitting model and parameters for each individual.
    """
    plt.figure(figsize=(12, 6))
    best_fits = {}

    for i, id in enumerate(id_pair):
        angles = df[df['idcollar'] == id]['turning_angle'].dropna().values

        # Histogram + KDE for visualization
        plt.subplot(1, 2, i + 1)
        plt.hist(angles, bins=30, density=True, alpha=0.6, color='gray', label='Observed')
        sns.kdeplot(angles, color='black', linewidth=1.5, label='KDE')

        # Compute mean turn angles
        mean_turn_angle = np.degrees(np.mean(angles))
        
        # Fit Mixture of Wrapped Normal Distributions
        wrapped_normal_params = fit_mixture_wrapped_normal(angles)

        # Compute persist_dir (small turns < 10°)
        threshold = np.radians(10)  # Convert 10 degrees to radians
        persist_dir = np.mean(np.abs(angles) < threshold)

        # Compute std_persist_turns from only small turning angles
        persist_turn_angles = angles[np.abs(angles) < threshold]
        std_persist_turns = np.degrees(np.std(persist_turn_angles))

        # Store best fit
        best_fits[id] = {"distribution": "Mixture of Wrapped Normals", 
                         "parameters": wrapped_normal_params,
                         "persist_dir": persist_dir,
                         "std_persist_turns": std_persist_turns,
                         "mean_turn_angle": mean_turn_angle
                        }

        # Plot best-fitting model
        x = np.linspace(-np.pi, np.pi, 100)
        w, mu1, sigma1, mu2, sigma2 = wrapped_normal_params
        y = (
            w * stats.vonmises.pdf(x, sigma1, loc=mu1) +
            (1 - w) * stats.vonmises.pdf(x, sigma2, loc=mu2)
        )

        plt.plot(x, y, label=f'Best fit: Mixture of Wrapped Normals', linewidth=2)
        plt.title(f'Turning Angle Distribution - {id}')
        plt.xlabel('Turning Angle (radians)')
        plt.ylabel('Density')
        plt.legend()

        # Print best-fitting distribution parameters
        print(f"Best fit for {id}: Mixture of Wrapped Normals")
        print(f"Parameters: {wrapped_normal_params}")
        print(f"Persist_dir for {id}: {persist_dir:.3f}, Std_persist_turns: {std_persist_turns:.3f}, Mean Turn Angle: {mean_turn_angle:.4f} degrees")

    plt.tight_layout()
    plt.show()

    return best_fits

In [13]:
def analyze_slope_distribution(df, id_pair):
    """
    Computes and plots the best-fit probability distributions for slope.
    """
    distributions = [stats.chi2, stats.gamma, stats.lognorm]
    best_fits = {}

    for id in id_pair:
        print(f"\n--- Analyzing Slope Distribution for {id} ---")
        best_fits[id] = {}

        values = df[df["idcollar"] == id]["slope"].dropna().values

        if len(values) == 0:
            print(f"Warning: No slope values found for {id}'s home range.")
            continue

        # Fit multiple distributions and select the best
        best_fit = None
        best_ll = -np.inf
        best_params = None

        plt.figure(figsize=(6, 4))
        plt.hist(values, bins=30, density=True, alpha=0.6, color='gray', label='Observed')

        for dist in distributions:
            try:
                params = dist.fit(values)
                ll = np.sum(dist.logpdf(values, *params))

                if ll > best_ll:
                    best_ll = ll
                    best_fit = dist
                    best_params = params

            except Exception as e:
                print(f"Could not fit {dist.name} for slope in {id}: {e}")

        # Store best fit
        if best_fit:
            best_fits[id]["slope"] = {"distribution": best_fit.name, "parameters": best_params}

            # Plot best-fitting distribution
            x = np.linspace(min(values), max(values), 100)
            y = best_fit.pdf(x, *best_params)
            plt.plot(x, y, label=f'Best fit: {best_fit.name}', linewidth=2)

        plt.title(f"Slope Distribution - {id}")
        plt.xlabel("Slope Value")
        plt.ylabel("Density")
        plt.legend()
        plt.show()

        # Print best-fitting distribution parameters
        print(f"Best fit for slope ({id}): {best_fit.name}")
        print(f"Parameters: {best_params}")

    return best_fits

In [14]:
def check_and_fit_prey_distribution(df, id_pair, threshold=0.6):
    """
    Checks the prey occupancy distribution and fits probability distributions for included prey types.

    Parameters:
    - df: DataFrame containing GPS tracking data
    - id_pair: List of individual IDs (e.g., [37821, 229032])
    - threshold: Minimum occupancy value required for inclusion

    Returns:
    - Dictionary with the best-fitting distribution and parameters for each included prey type per individual.
    """
    # Define candidate distributions
    distributions = [stats.gamma, stats.lognorm, stats.beta, stats.weibull_min]

    best_fits = {}

    for id in id_pair:
        print(f"\n--- Checking and Fitting Prey Distribution for {id} ---")
        best_fits[id] = {}

        plt.figure(figsize=(12, 4))

        for i, prey in enumerate(["sb", "bt", "gr"]):  # Include "gr" if relevant
            values = df[df["idcollar"] == id][prey].dropna()

            # Plot histogram
            plt.subplot(1, 3, i + 1)
            plt.hist(values, bins=30, color='gray', alpha=0.7, density=True)
            plt.axvline(threshold, color='red', linestyle='dashed', linewidth=2, label="Threshold (0.6)")
            plt.xlabel(f"{prey.capitalize()} Occupancy")
            plt.ylabel("Density")
            plt.title(f"{prey.capitalize()} Occupancy Distribution - {id}")
            plt.legend()

            # # Check if majority of values are below threshold
            # below_threshold = np.sum(values < threshold) / len(values)
            # if below_threshold > 0.5:  # If more than 50% of values are below 0.6, exclude this prey
            #     print(f"Excluding {prey} for {id} (majority occupancy < {threshold})")
            #     continue  # Skip fitting the distribution

            # print(f"Including {prey} for {id} (sufficient occupancy ≥ {threshold})")

            # Fit the best distribution
            best_fit = None
            best_ll = -np.inf
            best_params = None

            for dist in distributions:
                try:
                    params = dist.fit(values)
                    ll = np.sum(dist.logpdf(values, *params))

                    if ll > best_ll:
                        best_ll = ll
                        best_fit = dist
                        best_params = params

                except Exception as e:
                    print(f"Could not fit {dist.name} for {prey} in {id}: {e}")

            # Store best fit
            if best_fit:
                best_fits[id][prey] = {"distribution": best_fit.name, "parameters": best_params}

                # Plot best-fitting distribution
                x = np.linspace(min(values), max(values), 100)
                y = best_fit.pdf(x, *best_params)
                plt.plot(x, y, label=f'Best fit: {best_fit.name}', linewidth=2)

        plt.tight_layout()
        plt.show()

    return best_fits

In [ ]:
id_pair = [131343, 37821]  # IDs of tiger and leopard
file_path = "../../data/tiger_leopard_env_tg.csv"
output_path = f"../../data/adjusted_timeline_{id_pair[0]}_{id_pair[1]}.csv"

df_extracted = extract_adjusted_tracking_data(file_path, id_pair, output_path)

In [16]:
df_extracted.groupby(['idcollar'])[['Time_LMT']].agg(['min','max','count'])

Time_LMT                          
                         min                 max count
idcollar                                              
37821    2019-10-09 10:00:00 2020-03-28 03:00:00  3470
131343   2019-10-09 10:00:00 2020-03-28 03:00:00  3913

# Prepare Combined Multi-context Raster

In [17]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint, mapping
import scipy.stats as stats
from rasterio.mask import mask
from rasterio.features import rasterize

def clip_and_combine_probability(df, raster_paths, output_folder, id_pair, slope_fits, prey_fits, ssf_weights_dict):
    """
    Clips required raster layers to MCP, aligns them, converts to probability, applies weighted sum using individual SSF coefficients, 
    normalizes, saves the final combined raster, and returns the combined probability arrays.

    Parameters:
    - df: DataFrame containing GPS tracking data with 'proj_lon' and 'proj_lat'
    - raster_paths: Dictionary with raster file paths (keys include "slope", and any prey keys like "bt", "gr", etc.)
    - output_folder: Path to save clipped and combined rasters
    - id_pair: List of individual IDs (e.g., [37821, 229032])
    - slope_fits: Dictionary with best-fit slope distributions
    - prey_fits: Dictionary with best-fit prey distributions
    - ssf_weights_dict: Dictionary of SSF weights for each individual

    Returns:
    - combined_probs: Dictionary mapping each individual ID to its combined probability array.
    """ 

    combined_probs = {}
    prob_means = {}
    combined_masks = {}
    
    for id in id_pair:
        print(f"\nProcessing individual {id}...")

        # Get the SSF weights for this individual
        if id not in ssf_weights_dict:
            print(f"⚠ Warning: No SSF weights found for {id}. Skipping...")
            continue
        ssf_weights = ssf_weights_dict[id]

        # Filter only relevant environmental factors (slope or prey layers)
        included_layers = [layer for layer in ssf_weights if layer in raster_paths]

        if not included_layers:
            print(f"⚠ Skipping {id}: No relevant environmental factors (slope or prey layers).")
            continue  # Skip this individual

        # # Get included environmental layers
        # included_layers = ["slope"]  # Slope is always included
        # included_layers += list(prey_fits.get(id, {}).keys())  # Add only included prey

        # Create MCP (Minimum Convex Polygon)
        subset = df[df["idcollar"] == id]
        points = [tuple(xy) for xy in subset[["proj_lon", "proj_lat"]].values]
        mcp = MultiPoint(points).convex_hull
        mcp_gdf = gpd.GeoDataFrame(geometry=[mcp], crs="EPSG:32647")  # UTM Zone 47N

        clipped_rasters = {}

        for layer in included_layers:
            raster_path = raster_paths[layer]

            with rasterio.open(raster_path) as src:
                # Clip the raster to MCP
                clipped_image, clipped_transform = mask(src, mcp_gdf.geometry, crop=True)

                # Ensure correct raster shape
                if len(clipped_image.shape) == 3 and clipped_image.shape[0] == 1:
                    clipped_image = clipped_image[0]  # Remove singleton band dimension

                # Save metadata
                clipped_meta = src.meta.copy()
                clipped_meta.update({
                    "height": clipped_image.shape[0],  # Updated for correct shape
                    "width": clipped_image.shape[1],
                    "transform": clipped_transform
                })

                # Save clipped raster
                clipped_raster_path = f"{output_folder}/{layer}_clipped_MCP_{id}.tif"
                with rasterio.open(clipped_raster_path, "w", **clipped_meta) as dst:
                    dst.write(clipped_image, 1)

                print(f"Clipped {layer} raster (MCP) saved for {id}: {clipped_raster_path}")

                # Store clipped raster for alignment
                clipped_rasters[layer] = clipped_image

        # rasterize the MCP to a hard mask
        H, W = next(iter(clipped_rasters.values())).shape
        hull_mask = rasterize(
            [(mapping(mcp), 1)],
            out_shape=(H, W),
            transform=clipped_transform,
            fill=0,
            all_touched=True,
            dtype="uint8"
        )
        combined_masks[id] = hull_mask

        # Step 1: Find the Smallest Common Shape (Height & Width)
        min_height = min(image.shape[0] for image in clipped_rasters.values())
        min_width = min(image.shape[1] for image in clipped_rasters.values())

        # Step 2: Resize All Rasters to the Smallest Common Shape
        aligned_rasters = {layer: image[:min_height, :min_width] for layer, image in clipped_rasters.items()}

        # Step 3: Convert Cropped Rasters to Probability using SSF Weights
        combined_prob = np.zeros((min_height, min_width), dtype=np.float32)

        for layer, image in aligned_rasters.items():
            if layer not in ssf_weights:
                continue  # Skip layers without SSF weights

            # # Clip extreme values
            # image = np.clip(image, 0, np.percentile(image, 99))

            # Handle extreme values by clipping within a reasonable range
            if layer == "slope":
                if id in slope_fits and "slope" in slope_fits[id]:  # Ensure slope exists in slope_fits
                    dist_params = slope_fits[id]["slope"]["parameters"]
                    image = np.clip(image, 0, np.percentile(image, 99))  # Clip outliers
                    prob_layer = stats.lognorm.pdf(image, *dist_params)
                else:
                    print(f"⚠ Warning: No slope fit found for {id}, skipping slope layer.")
                    continue
        
            else:  # For prey layers (e.g., bt, gr)
                if id in prey_fits and layer in prey_fits[id]:  # Ensure layer exists in prey_fits
                    dist_params = prey_fits[id][layer]["parameters"]
                    image = np.clip(image, 0, np.percentile(image, 99))  # Clip extreme values
                    prob_layer = stats.beta.pdf(image, *dist_params)
                else:
                    print(f"⚠ Warning: No prey fit found for {id} - {layer}, skipping.")
                    continue

            # Fix: Remove invalid values (NaNs, negative values)
            prob_layer[np.isnan(prob_layer)] = 0
            prob_layer[prob_layer < 0] = 0

            # Fix: Normalize each layer before applying weights
            if np.nanmax(prob_layer) > 0:
                prob_layer = (prob_layer - np.nanmin(prob_layer)) / (np.nanmax(prob_layer) - np.nanmin(prob_layer))
            else:
                prob_layer[:] = 0  # If no valid values, set to zero

            # Fix: Apply log transformation to prevent extreme small values
            prob_layer = np.log1p(prob_layer)

            # Apply weight from SSF
            combined_prob += ssf_weights[layer] * prob_layer

        # Normalize final combined probability raster to 0-1
        if np.nanmax(combined_prob) > 0:
            combined_prob = (combined_prob - np.nanmin(combined_prob)) / (np.nanmax(combined_prob) - np.nanmin(combined_prob))
        else:
            combined_prob[:] = 0  # If no valid values, set to zero

        # Ensure no negative values
        combined_prob[combined_prob < 0] = 0

        # Ensure correct shape
        combined_prob = np.squeeze(combined_prob)
        if combined_prob.ndim == 2:
            combined_prob = combined_prob[np.newaxis, :, :]

        print("Final combined_prob shape:", combined_prob.shape)

        # Compute Mean Probability
        prob_mean = np.nanmean(combined_prob) if np.any(~np.isnan(combined_prob)) else 0
        prob_means[id] = prob_mean
        print(f"Mean probability (prob_mean) for {id}: {prob_mean:.3f}")

        # Save the final raster
        new_meta = clipped_meta.copy()
        new_meta.update({
            "height": combined_prob.shape[1],
            "width": combined_prob.shape[2],
            "count": 1
        })

        combined_output_path = f"{output_folder}/combined_probability_MCP_{id}.tif"
        with rasterio.open(combined_output_path, "w", **new_meta) as dst:
            dst.write(np.nan_to_num(combined_prob, nan=0).astype(rasterio.float32))

        print(f"Final combined probability raster saved for {id}: {combined_output_path}")

        combined_probs[id] = combined_prob

    return combined_probs, prob_means, combined_masks

# CRW

In [18]:
import math, random
import rasterio
from osgeo import gdal

In [19]:
def test_angle(angle):
    """
    Normalize any angle (in degrees) into the [0, 360) range.
    """
    return angle % 360

In [20]:
def compute_angle_distance_move(new_dir, dist):
    """
    Given a direction (in degrees) and a distance,
    compute the pixel offset (delta_x, delta_y) using a quadrant‐based method.
    Assumes a coordinate system where x increases to the right and y increases downward.
    """
    new_dir = test_angle(new_dir)
    if 0 <= new_dir <= 90:
        delta_x = round(dist * sin(radians(new_dir)))
        delta_y = -round(dist * cos(radians(new_dir)))
    elif 90 < new_dir <= 180:
        delta_x = round(dist * sin(radians(180 - new_dir)))
        delta_y = round(dist * cos(radians(180 - new_dir)))
    elif 180 < new_dir <= 270:
        delta_x = -round(dist * sin(radians(new_dir - 180)))
        delta_y = round(dist * cos(radians(new_dir - 180)))
    elif 270 < new_dir < 360:
        delta_x = -round(dist * sin(radians(360 - new_dir)))
        delta_y = -round(dist * cos(radians(360 - new_dir)))
    else:
        delta_x, delta_y = 0, 0
    return (delta_x, delta_y)

In [21]:
def create_cell_boundary_matrix():
    """
    Creates a point_list representing the eight adjacent cells.
    Two extra points (first two again) are appended to allow wrap-around indexing.
    """
    pl = point_list()
    # Define the eight neighbors (clockwise starting from top-left)
    neighbors = [(-1, -1), (0, -1), (1, -1),
                 (1,  0), (1,  1), (0,  1),
                 (-1, 1), (-1, 0)]
    for pt in neighbors:
        pl.append_point(Point(pt[0], pt[1]))
    # Append the first two points again for easy wrap-around indexing.
    for pt in neighbors[:2]:
        pl.append_point(Point(pt[0], pt[1]))
    return pl

In [22]:
class Point:
    """
    A simple Point class.
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def get_X(self):
        return self.x
    def get_Y(self):
        return self.y
    def get_coord(self):
        return (self.x, self.y)

In [23]:
class point_list():
    """
    A custom list to hold Point objects and provide convenience methods.
    """
    def __init__(self):
        self.points = []
    def get_length(self):
        return len(self.points)
    def get_Xs(self):
        return [pt.get_X() for pt in self.points]
    def get_Ys(self):
        return [pt.get_Y() for pt in self.points]
    def append_point(self, pt):
        self.points.append(pt)
    def get_point(self, n):
        return self.points[n]
    def plot_point_list_onRaster(self, myArray):
        plt.plot(self.get_Xs(), self.get_Ys(), 'ro-')
        plt.plot(self.get_point(0).get_X(), self.get_point(0).get_Y(), 'gs', label="Start")
        plt.plot(self.get_point(self.get_length()-1).get_X(), self.get_point(self.get_length()-1).get_Y(), 'ys', label="End")
        imgplot = plt.imshow(myArray, vmin=0, vmax=1024)
        imgplot.set_cmap('spectral')
        plt.title("Track on Raster")
        plt.legend()
        plt.show()
    def plot_point_list(self):
        plt.plot(self.get_Xs(), self.get_Ys(), 'ro-')
        plt.plot(self.get_point(0).get_X(), self.get_point(0).get_Y(), 'gs', label="Start")
        plt.plot(self.get_point(self.get_length()-1).get_X(), self.get_point(self.get_length()-1).get_Y(), 'ys', label="End")
        plt.title("Track")
        plt.legend()
        plt.show()

In [24]:
# This function reads a raster and converts it to a numpy array
def raster_to_numpy_array(rasterFile):
    # open raster dataset
    dem = gdal.Open(rasterFile)

    print ('Driver: ', dem.GetDriver().ShortName,'/', dem.GetDriver().LongName,)
    print ('Size is ',dem.RasterXSize,'x',dem.RasterYSize,'x',dem.RasterCount)
    print ('Projection is ',dem.GetProjection())
    geotransform = dem.GetGeoTransform()
    if not geotransform is None:
        print ('Origin = (',geotransform[0], ',',geotransform[3],')')
        print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')
    myArray = np.array(dem.GetRasterBand(1).ReadAsArray())

    return myArray, geotransform

In [25]:
# this function create a GeoTiff raster with specification of originalInputRaster
# and saves a numpy array to that GeoTiff
def numpy_array_to_raster(myArray,outputFile, originalInputRaster):
    inRaster = gdal.Open(originalInputRaster)

    # get parameters
    geotransform = inRaster.GetGeoTransform()
    spatialreference = inRaster.GetProjection()
    ncol = inRaster.RasterXSize
    nrow = inRaster.RasterYSize
    nband = 1
    # create dataset for output
    fmt = 'GTiff'
    driver = gdal.GetDriverByName(fmt)
    dst_dataset = driver.Create(outputFile, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(myArray)
    dst_dataset = None

In [26]:
def identify_slice(n_dir):
       # ******************  select three closest pixels in n_dir ***********************
        num_slices = 8
        seg_size = 360 / num_slices
        range_1 = - seg_size / 2.0 + 360
        range_2 = (range_1 + seg_size) - 360
        slice_ind = 0
        if (n_dir >= range_1) or (n_dir <= range_2):
            slice_ind = 1
        # print("slice ",slice)
        else:
            # (find which segment the new_dir is in
            # print(" **** In else loop ****")
            for p in range(num_slices - 1):
                range_1 += seg_size
                range_2 += seg_size
                # print("Range in for loop ",range_1,range_2)
                if range_1 > 360:
                    range_1 -= 360
                # print("range 1 & 2",range_1,range_2)
                if range_1 < n_dir <= range_2:
                    slice_ind = p + 2
                    break
        # print("slice, new direction", slice,n_dir)
        return slice_ind

In [27]:
def animating_track(x_coord,y_coord, back_raster):
    [cols,rows]= back_raster.shape
    x = len(x_coord)
    # animation code
    # plt.subplot(1,2,1)
    fig = plt.figure()
    ax = plt.axes(xlim=(0, rows), ylim=(cols,0))
    line, = ax.plot([], [], 'wo',linewidth=0)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        # line.set_data(i,i)
        line.set_data(x_coord[1:i], y_coord[1:i])
        return line,

    # use x for number of points/frames
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=x, interval=50, blit=True)
    imgplot = ax.imshow(back_raster, cmap="viridis", vmin=0, vmax=np.nanmax(back_raster))
    # imgplot.set_cmap('spectral')
    plt.show()
    print("Track animation done!")

In [28]:
def crw_combined(num_pts, persis_dir, mu, sigma, sigma_1, 
                 myArray, cols, rows, envArray, hull_mask, mat_use, 
                 time_step, start_time, 
                 id_selected, best_step_length_fits, best_turning_angle_fits,
                 max_tries_per_step = 50
                ):
    """
    Simulated CRW that uses the combined probability raster (envArray) as context.
    """

    print("\nInitializing CRW simulation...")

    # Get movement parameters for the selected individual
    step_params = best_step_length_fits[id_selected]["parameters"]
    turn_params = best_turning_angle_fits[id_selected]["parameters"]
    mix_p, mu1, kappa1, mu2, kappa2 = turn_params
    
    # Choose a random starting point that is valid (within myArray range)
    valid = False
    while not valid:
        new_x = np.random.randint(0, cols)
        new_y = np.random.randint(0, rows)
        if (0 <= new_x < cols and 0 <= new_y < rows and 
            0 <= myArray[new_y, new_x] <= 1 and 
            hull_mask[new_y, new_x] == 1 and
            envArray[new_y, new_x] > 0):
            valid = True
     
    x_coord = [new_x]
    y_coord = [new_y]
    mat_use[new_y, new_x] += 1
    
    # Set initial direction (random 0-360 degrees)
    prev_dir = random.uniform(0, 360)

    print(f"Starting simulation at ({new_x}, {new_y})")
    print("Entering main simulation loop...")

    test_pts = 0  # count of simulated steps
    
    while test_pts <= num_pts:
        test_pts += 1

        if test_pts % 1000 == 0:  # Print every 1000 steps to track progress
            print(f"Step {test_pts}/{num_pts} - Current position: ({new_x}, {new_y})")

        retries = 0
        success = False
        while retries < max_tries_per_step and not success:
            retries += 1
            
            # --- Determine new direction ---
            if random.random() < persis_dir:
                # new_dir = prev_dir + np.random.vonmises(mu2, sigma2)
                new_dir = (prev_dir + np.random.normal(mu, sigma))
            else:
                # new_dir = prev_dir + np.random.vonmises(mu3, sigma3)
                new_dir = (prev_dir + np.random.normal(mu, sigma_1))
            new_dir = test_angle(new_dir)  # adjust to [0,360)
            
            # --- Sample a step length from the fitted lognorm distribution ---
            # Using np.random.lognormal with parameters: mean and sigma of underlying normal.
            step_km = np.random.lognormal(step_params[0], step_params[2]) # increase variation
            step_m  = step_km * 1000
            max_m   = 300 * time_step # 0.3 km/h
            step_m  = min(step_m, max_m)            
    
            # # Limit max step length
            pixel_size = 30
            step_pixels = int(round(step_m / pixel_size))

            # print(f"Step {test_pts}: Step length = {step:.2f}m ")
            
            # Compute endpoint using the step length and new direction (as a vector)
            delta_x, delta_y = compute_angle_distance_move(new_dir, step_pixels)
            t_x = new_x + delta_x
            t_y = new_y + delta_y
        
            # Check if endpoint is valid in DEM (myArray)
            if (t_x < 0 or t_x >= cols or t_y < 0 or t_y >= rows or 
                myArray[t_y, t_x] < 0 or myArray[t_y, t_x] > 1 or  hull_mask[t_y, t_x] == 0 or
                envArray[t_y, t_x] <= 0 or np.isnan(envArray[t_y, t_x])):
                retries += 1
                # print(f"Step {test_pts}: Invalid position ({t_x}, {t_y}), retrying...")
                # If invalid, reinitialize direction randomly and retry this step
                # **Limit retries per step to prevent infinite loops**
                if retries > 100:
                    # print(f"    Too many retries at step {test_pts}, adjusting strategy.")
                    prev_dir = np.random.random() * 360  # Choose a random direction to escape
                    retries = 0  # Reset retry count
                success = False
                continue  # Skip this step

            # success >= accept
            success = True
            new_x, new_y = t_x, t_y
            x_coord.append(new_x)
            y_coord.append(new_y)
            mat_use[new_y, new_x] += 1

            # update direction
            prev_dir = new_dir
            retries = 0

        if not success:
            pass
            
        # End of one step: update current position to last new_x, new_y
        # (They are already updated in the loop.)
    print("CRW Done! number of points =", len(x_coord))
    return mat_use, x_coord, y_coord #, time_list

In [29]:
def plot_visit_matrix(visit_matrix, geotransform, title="Visitation Matrix"):
    """Plots the visitation matrix."""
    plt.figure(figsize=(8, 6))
    plt.imshow(visit_matrix, cmap="hot", interpolation="nearest", origin="upper")
    plt.colorbar(label="Visit Count")
    plt.title(title)
    plt.xlabel("X Pixels")
    plt.ylabel("Y Pixels")
    plt.show()

In [30]:
def compute_observed_visit_matrix(df_indiv, geotransform, rows, cols):
    """Computes visit matrix from observed tracking data."""
    visit_matrix = np.zeros((rows, cols), dtype=np.int64)
    
    for _, row in df_indiv.iterrows():
        x, y = row['proj_lon'], row['proj_lat']
        col_idx = int((x - geotransform[0]) / geotransform[1])
        row_idx = int((geotransform[3] - y) / abs(geotransform[5]))

        # Skip out-of-bounds points
        if 0 <= row_idx < rows and 0 <= col_idx < cols:
            visit_matrix[row_idx, col_idx] += 1  
        else:
            print(f"⚠ Skipping out-of-bounds point: ({x}, {y}) → (row={row_idx}, col={col_idx})")
            
    return visit_matrix

In [31]:
def plot_side_by_side_visitation_matrices(observed_matrix, simulated_matrix, M_C):
    """
    Plots observed and simulated visit matrices side by side for comparison.
    
    - The highest 10% of values (95th percentile and above) are displayed as the same color (white).
    - The simulated matrix is divided by M_C (number of Monte Carlo runs) to ensure comparability.
    - A consistent color scale is applied to both matrices.
    """
    
    # Normalize simulated matrix by dividing by M_C
    simulated_matrix = simulated_matrix / M_C

    # Extract nonzero values to compute color scale
    observed_nonzero = observed_matrix[observed_matrix > 0]
    simulated_nonzero = simulated_matrix[simulated_matrix > 0]

    # Compute the 95th percentile to avoid extreme outliers dominating
    observed_p95 = np.percentile(observed_nonzero, 95) if len(observed_nonzero) > 0 else 1
    simulated_p95 = np.percentile(simulated_nonzero, 95) if len(simulated_nonzero) > 0 else 1

    # Use the same color scale for both matrices based on the highest 95th percentile
    color_max = max(observed_p95, simulated_p95, 1)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Observed Visit Matrix with adjusted color scaling
    im1 = axes[0].imshow(observed_matrix, cmap="hot", interpolation="nearest", origin="upper",
                         vmin=0, vmax=color_max)
    axes[0].set_title("Observed Visitation Matrix")
    cbar1 = plt.colorbar(im1, ax=axes[0])
    cbar1.set_label("Visit Count")

    # Simulated Visit Matrix with adjusted color scaling
    im2 = axes[1].imshow(simulated_matrix, cmap="hot", interpolation="nearest", origin="upper",
                         vmin=0, vmax=color_max)
    axes[1].set_title("Simulated Visitation Matrix")
    cbar2 = plt.colorbar(im2, ax=axes[1])
    cbar2.set_label("Visit Count (Averaged)")

    # Labels
    for ax in axes:
        ax.set_xlabel("X Pixels")
        ax.set_ylabel("Y Pixels")

    plt.tight_layout()
    plt.show()

In [32]:
def plot_boxplot_and_test(observed_matrix, simulated_matrix, M_C):
    """
    Plots a boxplot comparing observed and simulated visitation counts 
    and performs a statistical test (Mann-Whitney U test).
    """

    # # Compute the scaling factor for the simulated data
    # scaling_factor =  simulated_total_pts / observed_total_pts 
    # print(f"Scaling factor: {scaling_factor:.4f}")

    # Scale the simulated visitation counts
    scaled_simulated_visitation = simulated_matrix / M_C

    # Flatten matrices to extract nonzero visitation counts
    observed_counts = observed_matrix.flatten()
    simulated_counts = scaled_simulated_visitation.flatten()

    # Remove zero values (cells that were never visited)
    observed_counts = observed_counts[observed_counts > 0]
    simulated_counts = simulated_counts[simulated_counts > 0]

    # Remove NaN values before performing statistical tests**
    observed_counts = observed_counts[~np.isnan(observed_counts)]
    simulated_counts = simulated_counts[~np.isnan(simulated_counts)]

    # **Ensure observed and simulated counts have the same length without NaNs**
    min_len = min(len(observed_counts), len(simulated_counts))
    observed_counts = observed_counts[:min_len]  # Trim to shortest length
    simulated_counts = simulated_counts[:min_len]  # Trim to shortest length

    # Convert to DataFrame for easier boxplotting
    df_boxplot = pd.DataFrame({
        "Observed": observed_counts,
        "Simulated": simulated_counts
    })
    print(f"Observed count size: {len(observed_counts)}")
    print(f"Simulated count size: {len(simulated_counts)}")
    print("Unique values in observed:", np.unique(observed_counts))
    print("Unique values in simulated:", np.unique(simulated_counts))
    
    # Plot boxplot comparing visitation distributions
    plt.figure(figsize=(8, 6))
    df_boxplot.boxplot(column=["Observed", "Simulated"], showfliers=True)
    # plt.yscale("log")  # Log scale to handle skewness in visitation data
    plt.ylabel("Visitation Count")
    plt.title("Comparison of Visitation Distributions (Observed vs. Simulated)")
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.show()

    # --- Perform Mann-Whitney U Test ---
    if len(observed_counts) > 0 and len(simulated_counts) > 0:
        stat, p_value = mannwhitneyu(observed_counts, simulated_counts, alternative='two-sided')
        print(f"🔍 Mann-Whitney U test results:")
        print(f"   U-statistic: {stat:.3f}")
        print(f"   p-value: {p_value:.3e}")
        if p_value < 0.05:
            print("   ➡ Significant difference detected between observed and simulated visitations (p < 0.05).")
        else:
            print("   ➡ No significant difference detected (p ≥ 0.05).")
    else:
        print("⚠ Skipping statistical test: One or both distributions are empty.")

In [33]:
def create_crw_monte_carlo_simulation(M_C, num_pts, persis_dir, mu, sigma, sigma_1, 
                                      myArray, cols, rows, envArray, hull_mask, mat_use, time_step, 
                                      start_time, id_selected, best_step_length_fits, best_turning_angle_fits):
    
    all_simulated_data = []  # Store all MC simulation trajectories
    # For this example, we run one MC simulation; you can loop over M_C if needed.
    for mc_round in range(1, M_C + 1):
        print(f"Beginning Monte Carlo iteration {mc_round}/{M_C}")
        current_mat_use, xc, yc = crw_combined(num_pts, persis_dir, mu, sigma, sigma_1, 
                                               myArray, cols, rows, envArray, hull_mask, mat_use, time_step,
                                               start_time, id_selected, best_step_length_fits, best_turning_angle_fits)
        # Accumulate visitation matrix across MC runs
        mat_use += current_mat_use

        if len(xc) > 1:
            # total time for the entire track
            total_duration_hours = num_pts * time_step
            # We'll assign times linearly from start_time to start_time + total_duration_hours
            time_list = [
                start_time + pd.Timedelta(hours=(total_duration_hours * i/(len(xc)-1)))
                for i in range(len(xc))
            ]
        else:
            time_list = [start_time] * len(xc)
            
        # Store trajectory with MC round information
        df_mc = pd.DataFrame({
            "idcollar": [id_selected] * len(xc),
            "x": xc,
            "y": yc,
            "time": time_list,
            "MC_round": [mc_round] * len(xc)
        })
        
        all_simulated_data.append(df_mc)

    # Combine all Monte Carlo trajectories into a single DataFrame
    df_all_mc = pd.concat(all_simulated_data, ignore_index=True)
    
    return mat_use, df_all_mc

In [34]:
def extract_from_clipped_raster(df, clipped_raster_path, geotransform):
    """
    Extracts environmental values from a clipped raster based on x, y pixel locations.
    Also converts pixel indices to projected coordinates.

    Parameters:
    - df: DataFrame with x, y pixel indices
    - clipped_raster_path: File path to the individual's clipped raster
    - geotransform: Raster geotransform to convert pixel indices to projected coordinates

    Returns:
    - List of extracted values
    - Lists of projected longitude and latitude
    """
    try:
        with rasterio.open(clipped_raster_path) as src:
            raster_data = src.read(1)  # Read the first band
            nodata_val = src.nodata  # NoData value

            extracted_values = []
            proj_lons = []
            proj_lats = []

            for _, row in df.iterrows():
                x_pix, y_pix = int(row["x"]), int(row["y"])  # Convert to integer pixel indices

                if 0 <= x_pix < src.width and 0 <= y_pix < src.height:
                    value = raster_data[y_pix, x_pix]

                    # Handle NoData values
                    if nodata_val is not None and value == nodata_val:
                        value = np.nan

                    extracted_values.append(value)

                    # Convert pixel index to projected coordinates
                    lon = geotransform[0] + x_pix * geotransform[1]
                    lat = geotransform[3] + y_pix * geotransform[5]
                    proj_lons.append(lon)
                    proj_lats.append(lat)
                else:
                    extracted_values.append(np.nan)
                    proj_lons.append(np.nan)
                    proj_lats.append(np.nan)

            return extracted_values, proj_lons, proj_lats
    except Exception as e:
        print(f"⚠ Error reading {clipped_raster_path}: {e}")
        return [np.nan] * len(df), [np.nan] * len(df), [np.nan] * len(df)  # Return NaNs if raster cannot be read


# Interaction Analysis

# Reverse slicing for every 1000 hours (Tail to Head)

In [36]:
def print_time_step_diagnostics(df_slice, id_pair, slice_idx, win_start=None, win_end=None):
    if win_start and win_end:
        print(f"\n[Slice {slice_idx:04d}] window {win_start} → {win_end}")
    else:
        print(f"\n[Slice {slice_idx:04d}] Time Interval Diagnostics:")

    for id_sel in id_pair:
        df_indiv = df_slice[df_slice["idcollar"] == id_sel].sort_values("time")
        df_indiv["time"] = pd.to_datetime(df_indiv["time"], errors="coerce")
        time_deltas = df_indiv["time"].diff().dropna()
        n_points = len(df_indiv)
        if not time_deltas.empty:
            median_step = time_deltas.median().total_seconds() / 60.0  # in minutes
            print(f"  ID {id_sel}: n = {n_points} | median = {median_step:.2f} min | "
                  f"min = {time_deltas.min()} | max = {time_deltas.max()}")
        else:
            print(f"  ID {id_sel}: n = {n_points} | Not enough data")

In [ ]:
from pathlib import Path
import pandas as pd

print(id_pair)
file_path = "../../data/tiger_leopard_env_tg.csv"

# 1) read & slice your *full* calibrated CSV
df_full = extract_adjusted_tracking_data(file_path, id_pair, 
                                         f"../../data/adjusted_timeline_{id_pair[0]}_{id_pair[1]}.csv")
df_full["Time_LMT"] = pd.to_datetime(df_full["Time_LMT"])
df_full = df_full.rename(columns={"Time_LMT":"time"})

# Convert 'time' to datetime format (this fixes the issue)
df_full["time"] = pd.to_datetime(df_full["time"], errors="coerce")

# Now, apply the correct formatting (if needed)
df_full["time"] = df_full["time"].dt.strftime('%Y-%m-%d %H:%M:%S')

df_full["time"] = pd.to_datetime(df_full["time"], errors="coerce")
df_full.head()


# 1 000‑hour windows anchored at the end so every slice is full length
overall_start = df_full["time"].min()
overall_end   = df_full["time"].max()
window        = pd.Timedelta(hours=1000)

# build reverse windows ending at overall_end
rev_windows = []
cur_end = overall_end
while True:
    cur_start = cur_end - window
    if cur_start < overall_start:
        # head‐chunk shorter than 1000 h → drop
        break
    rev_windows.append((cur_start, cur_end))
    cur_end = cur_start

# reverse to chronological so slice000 is earliest
windows = list(reversed(rev_windows))
n_slices = len(windows)
print(f"Total slices of 1000 h each: {n_slices}")

# diagnostics
for i, (start, end) in enumerate(windows):
    s1 = df_full[(df_full["idcollar"] == id_pair[0]) &
                 (df_full["time"] >= start) &
                 (df_full["time"] <  end)]
    s2 = df_full[(df_full["idcollar"] == id_pair[1]) &
                 (df_full["time"] >= start) &
                 (df_full["time"] <  end)]
    df_slice = pd.concat([s1, s2], ignore_index=True)
    print_time_step_diagnostics(df_slice, id_pair, i, start, end)


In [ ]:
# Define file paths
raster_paths = {
    "slope": "D:/Tiger/Map/Environmental data/DEM/slope_wefcom_UTM47N.tif",
    "sb": "D:/Tiger/Map/Environmental data/Prey/Occupancy Data/sb1kmPredictiveMap_UTM47N.tif",
    "bt": "D:/Tiger/Map/Environmental data/Prey/Occupancy Data/bt1kmPredictiveMap_UTM47N.tif",
    "gr": "D:/Tiger/Map/Environmental data/Prey/Occupancy Data/gr1kmPredictiveMap_UTM47N.tif"
}

ssf_weights_dict = {
    31898: {"bt": 0.4572, "cos_ta": -0.0475},
    # 31899: {"cos_ta": -0.1140},
    37821: {"bt": 0.3930, "gr": 0.4409, "slope": -0.1226, "cos_ta": -0.2147},
    # 37822: {"cos_ta": 0.1180},
    37823: {"bt": 0.4614, "gr": 0.4045, "slope": -0.2446, "cos_ta": 0.1035},
    131343: {"bt": 0.7164, "slope": -0.0826},
    229011: {"slope": -0.0543, "cos_ta": -0.1512},
    229012: {"bt": 0.7511, "cos_ta": -0.1318, "slope": -0.0871},
    229022: {"slope": -2291, "cos_ta": -0.0828},
    229032: {"bt": 0.7476, "cos_ta": -0.0328, "slope": -0.1160},
    229041: {"bt": 0.7659, "slope": -0.2764}
}


In [ ]:
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
import workers_null_model

id_pair = [131343, 37821]
slice_output_base = Path(f"../../simulated_trajectory/CRW_slice_RT_MC1000/by_slice_{id_pair[0]}_{id_pair[1]}_reverse_time")
file_path = "../../data/tiger_leopard_env_tg.csv"

# 1) read & slice your *full* calibrated CSV
df_full = extract_adjusted_tracking_data(file_path, id_pair, 
                                         f"../../data/adjusted_timeline_{id_pair[0]}_{id_pair[1]}.csv")
df_full["Time_LMT"] = pd.to_datetime(df_full["Time_LMT"])
df_full = df_full.rename(columns={"Time_LMT":"time"})

# Convert 'time' to datetime format (this fixes the issue)
df_full["time"] = pd.to_datetime(df_full["time"], errors="coerce")

# Now, apply the correct formatting (if needed)
df_full["time"] = df_full["time"].dt.strftime('%Y-%m-%d %H:%M:%S')

df_full["time"] = pd.to_datetime(df_full["time"], errors="coerce")
df_full.head()


# 1 000‑hour windows anchored at the end so every slice is full length
overall_start = df_full["time"].min()
overall_end   = df_full["time"].max()
window        = pd.Timedelta(hours=1000)

# build reverse windows ending at overall_end
rev_windows = []
cur_end = overall_end
while True:
    cur_start = cur_end - window
    if cur_start < overall_start:
        # head‐chunk shorter than 1000 h → drop
        break
    rev_windows.append((cur_start, cur_end))
    cur_end = cur_start

# reverse to chronological so slice000 is earliest
windows = list(reversed(rev_windows))
n_slices = len(windows)
print(f"Total slices of 1000 h each: {n_slices}")

# diagnostics
for i, (start, end) in enumerate(windows):
    s1 = df_full[(df_full["idcollar"] == id_pair[0]) &
                 (df_full["time"] >= start) &
                 (df_full["time"] <  end)]
    s2 = df_full[(df_full["idcollar"] == id_pair[1]) &
                 (df_full["time"] >= start) &
                 (df_full["time"] <  end)]
    df_slice = pd.concat([s1, s2], ignore_index=True)
    print_time_step_diagnostics(df_slice, id_pair, i, start, end)
    
# Log‐spaced (hour→day→week) bins in minutes
time_bins   = [0, 60, 180, 360, 720, 1440, 2880, 5760, 11520, 23040, 30240]
time_labels = ["0-1h","1-3h","3-6h","6-12h","12-24h","1-2d","2-4d","4-8d","8-16d","16-21d"]

MC_START = 901 # revise
MC_END = 1000 # revise
tasks = []
for slice_idx, (win_start, win_end) in enumerate(windows):
    tasks.append((
        slice_idx,
        win_start,            # new argument
        win_end,              # new argument
        df_full,
        id_pair,
        raster_paths,
        slice_output_base,
        1000,                    # M_C
        time_bins,
        time_labels,
        MC_START,
        MC_END
    ))

n_workers = min(os.cpu_count()-1, len(tasks))

with ProcessPoolExecutor(max_workers=n_workers) as exe:
    futures = {
        exe.submit(workers_null_model.process_slice_reverse_time, *args): args[0]
        for args in tasks
    }

    for fut in as_completed(futures):
        slice_idx = futures[fut]
        try:
            sl, tot_ev, tot_pr = fut.result()
            print(f"[Slice {sl:04d}] ✅ done — events: {tot_ev}, pairs: {tot_pr}")
        except Exception as e:
            print(f"[Slice {slice_idx:04d}] ⚠ failed:", e)